In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
%matplotlib inline
plt.style.use('ggplot')
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer, OneHotEncoder, RobustScaler, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from xgboost import XGBRegressor

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def split(df, labels):
    return df.loc[:,labels]

In [20]:
def custom_cat(df):
    df = df.replace(['Yes', 'No'], [1,0]).fillna(0)
    df = df.replace(['Main campus', 'Not main campus'], [1,0]).fillna(0)
    
    df['custom__academics_num_bach'] = df.filter(like='_bach').sum(axis=1).astype('int')
    
    df['custom__school_campus'] = df.school__main_campus.isin(['Main campus']).astype('int')
    
    df['custom__religious_high'] = df.school__religious_affiliation.isin(['myw','cxp','pdf','aiy','thg','zug','sdh','bmv','fxo','qys',
                                             'nnm','onn','ibe','ntl','smi','aai','huu','mix','dpu','fuf','dqz','hmn',
                                             'xds','qzo','mky','hap','fiy','gju','lrj','emi','ddx','jqf']).astype('int')
    df['custom__religious_low'] = df.school__religious_affiliation.isin(['wxa', 'prn', 'qyb', 'nhu','uac', 'rgp', 'iqp',]).astype('int')
    df = df.drop(list(df.filter(like='school__religious_aff')), axis=1)
        
    df['custom__school_state_low'] = df.school__state.isin(['tus', 'nni','noz','ugr','aku','kta','qbv','iju','msx','qid','fen','bbk','sbh','uod',
                                           'gai','idl','gzi','xfa','qua','yyg','xtb','dlg','pgp','krj','bxo','zms','ste',]).astype('int')
    df['custom__school_state_high'] = df.school__state.isin(['prq', 'mig', 'tdb', 'iya', 'wzk', 'afu', 'iyc', 'exw', 'npw', 'rmt','jor','cyf','cmn','ncw','usz',
                                           'tlt','kho','xhl','dhx','nja','ony','rbl','xgy','fyo','das','fjm','hgy']).astype('int')
    df = df.drop(list(df.filter(like='school__state')), axis=1)
    
    df['custom__school_region_low'] = df.school__region_id.isin(['Southwest (AZ, NM, OK, TX)', 
                                 'Southeast (AL, AR, FL, GA, KY, LA, MS, NC, SC, TN, VA, WV)',
                                 'Outlying Areas (AS, FM, GU, MH, MP, PR, PW, VI)',
                                ]).astype('int')
    df['custom__school_region_high'] = df.school__region_id.isin(['New England (CT, ME, MA, NH, RI, VT)',
                                'Mid East (DE, DC, MD, NJ, NY, PA)',
                                'Plains (IA, KS, MN, MO, NE, ND, SD)',
                                'U.S. Service Schools']).astype('int')
    df = df.drop(list(df.filter(like='school__region')), axis=1)
    
    df['custom__carnegie_undergrad_four_year'] = df.school__carnegie_undergrad.str.contains('Four', na=False).astype('int')
    df['custom__carnegie_undergrad_two_year'] = df.school__carnegie_undergrad.str.contains('Two', na=False).astype('int')
    df['custom__carnegie_undergrad_low_transfer'] = df.school__carnegie_undergrad.str.contains('lower', na=False).astype('int')
    df['custom__carnegie_undergrad_high_transfer'] = df.school__carnegie_undergrad.str.contains('higher', na=False).astype('int')
    df['custom__carnegie_undergrad_part_time'] = df.school__carnegie_undergrad.str.contains(' part', na=False).astype('int')
    df['custom__carnegie_undergrad_full_time'] = df.school__carnegie_undergrad.str.contains(' full', na=False).astype('int')   
    df['custom__carnegie_undergrad_inclusive'] = df.school__carnegie_undergrad.str.contains('inclusive', na=False).astype('int')
    df['custom__carnegie_undergrad_selective'] = df.school__carnegie_undergrad.str.contains('selective', na=False).astype('int')

    df['custom__carnegie_undergrad_low'] = df.school__carnegie_undergrad.isin(['Four-year, full-time, inclusive, lower transfer-in', 
                                         'Four-year, higher part-time',
                                         'Four-year, medium full-time, inclusive, lower transfer-in',
                                         'Two-year, medium full-time',
                                         'Two-year, higher full-time',
                                         'Not applicable',
                                         'Two-year, mixed part/full-time',
                                         'Four-year, medium full-time, inclusive, higher transfer-in',
                                         'Two-year, higher part-time',]).astype('int')
    df['custom__carnegie_undergrad_high'] = df.school__carnegie_undergrad.isin(['Four-year, full-time, more selective, lower transfer-in',
                                         'Four-year, full-time, more selective, higher transfer-in',
                                         'Four-year, full-time, selective, higher transfer-in',
                                         'Four-year, full-time, selective, lower transfer-in',
                                         'Four-year, medium full-time, selective, lower transfer-in',
                                         'Four-year, medium full-time, selective, higher transfer-in',
                                         'Not classified (Exclusively Graduate)']).astype('int')
    df = df.drop('school__carnegie_undergrad', axis=1)
    
    df['custom__school_carnegie_size_setting_high'] = df.school__carnegie_size_setting.isin(['Four-year, large, highly residential',
                                             'Four-year, large, primarily residential',
                                             'Four-year, medium, highly residential',
                                             'Four-year, medium, primarily residential',
                                             'Four-year, small, highly residential',
                                             'Exclusively graduate/professional',
                                             'Four-year, small, primarily residential'
                                         ]).astype('int')
    df['custom__school_carnegie_size_setting_low'] = df.school__carnegie_size_setting.isin(['Four-year, very small, primarily nonresidential',
                                             'Not applicable',
                                             'Two-year, large',
                                             'Two-year, medium',
                                             'Two-year, small',
                                             'Two-year, very large',
                                             'Two-year, very small'
                                         ]).astype('int')
    df['custom__carnegie_size_large'] = df.school__carnegie_size_setting.str.contains('large', na=False).astype('int')
    df['custom__carnegie_size_medium'] = df.school__carnegie_size_setting.str.contains('medium', na=False).astype('int')
    df['custom__carnegie_size_small'] = df.school__carnegie_size_setting.str.contains('small', na=False).astype('int')
    df['custom__carnegie_size_highly_residential'] = df.school__carnegie_size_setting.str.contains('highly', na=False).astype('int')
    df['custom__carnegie_size_primarily_residential'] = df.school__carnegie_size_setting.str.contains('primarily', na=False).astype('int')
    df = df.drop('school__carnegie_size_setting', axis=1)
    
    df['custom__carnegie_basic_high'] = df.school__carnegie_basic.isin(['Baccalaureate Colleges: Arts & Sciences Focus',
                                      'Doctoral Universities: Highest Research Activity',
                                      "Master's Colleges & Universities: Larger Programs",
                                      'Doctoral Universities: Higher Research Activity',
                                      "Master's Colleges & Universities: Medium Programs",
                                      'Special Focus Four-Year: Medical Schools & Centers',
                                      "Master's Colleges & Universities: Small Programs",
                                      "Doctoral Universities: Moderate Research Activity",
                                      "Special Focus Four-Year: Law Schools",
                                      "Special Focus Four-Year: Engineering Schools",                                      
                                         ]).astype('int')
    df['custom__carnegie_basic_low'] = df.school__carnegie_basic.isin(['Not applicable',
                                      "Baccalaureate/Associate's Colleges: Mixed Baccalaureate/Associate's",
                                      "Special Focus Two-Year: Health Professions",
                                      "Baccalaureate/Associate's Colleges: Associate's Dominant",
                                      "Associate's Colleges: High Vocational & Technical-High Nontraditional",
                                      "Special Focus Four-Year: Other Technology-Related Schools",
                                      "Associate's Colleges: High Vocational & Technical-Mixed Traditional/Nontraditional",
                                      "Special Focus Two-Year: Other Fields",
                                      "Special Focus Four-Year: Business & Management Schools",
                                      "Associate's Colleges: Mixed Transfer/Vocational & Technical-Mixed Traditional/Nontraditional"
                                     ]).astype('int')
    df['custom__carnegie_basic_bacc'] = df.school__carnegie_basic.str.contains('Baccalaureate', na=False).astype('int')
    df['custom__carnegie_basic_special'] = df.school__carnegie_basic.str.contains('Special', na=False).astype('int')
    df['custom__carnegie_basic_assoc'] = df.school__carnegie_basic.str.contains('Associate', na=False).astype('int')
    df['custom__carnegie_basic_masters'] = df.school__carnegie_basic.str.contains('Master', na=False).astype('int')
    df['custom__carnegie_basic_doc'] = df.school__carnegie_basic.str.contains('Doctoral', na=False).astype('int')
    df = df.drop('school__carnegie_basic', axis=1)
    
    df['custom__school_online'] = df.school__online_only.isin(['Not distance-education only']).astype('int')
    df = df.drop('school__online_only', axis=1)
    
    df['custom__school_locale'] = df.school__locale.isin(['Suburb: Large (outside principal city, in urbanized area with population of 250,000 or more)',
                                            'City: Large (population of 250,000 or more)',
                                            'City: Midsize (population of at least 100,000 but less than 250,000)',
                                            'Rural: Fringe (rural territory up to 5 miles from an urbanized area or up to 2.5 miles from an urban cluster)']).astype('int')
    df['custom__school_locale_city'] = df.school__locale.str.contains('City', na=False).astype('int')
    df['custom__school_locale_suburb'] = df.school__locale.str.contains('Suburb', na=False).astype('int')
    df['custom__school_locale_rural'] = df.school__locale.str.contains('Rural', na=False).astype('int')
    df['custom__school_locale_town'] = df.school__locale.str.contains('Town', na=False).astype('int')
    df = df.drop('school__locale', axis=1)
    
    bachelors = ['biological', 'communication', 'computer', 'education', 'health', 'history', 'mathematics', 'humanities', 'language', 
    'multidiscipline', 'philosophy_religious', 'physical_science', 'psychology', 'social_science', 'visual_performing']
    bachelors = ['academics__program_bachelors_{}'.format(i) for i in bachelors]
    temp = df.loc[:,bachelors] == 1
    df['custom__academics_program_bachelors_good'] = temp.sum(axis=1)
    df['custom__academics_program_bachelors_business'] = df.academics__program_bachelors_business_marketing.isin([1]).astype('int')

    df = df.drop(list(df.filter(like='academics__program')), axis=1)
    df = df.drop('report_year', axis=1)
    
    df = pd.get_dummies(df, dummy_na=True)
    df['custom__sum_missing'] = df.loc[:,list(df.filter(like='nan'))].sum(axis=1)
    df = df.drop(list(df.filter(like='nan')), axis=1)
    #df = df.drop(list(df.filter(like='minority_serving')), axis=1)


    return df

In [4]:
def custom_num(df):
    log_cols = ['cost__title_iv_private_by_income_level_75001_110000', 'cost__title_iv_public_by_income_level_110001_plus',
                'completion__completion_cohort_4yr_100nt', 'cost__title_iv_private_by_income_level_75001_110000']
    for col in log_cols:
        df['custom__log_{}'.format(col)] = df[col].map(lambda x: np.log(x+1))
    
    log_percentage = ['academics__program_percentage_english', 'academics__program_percentage_history', 
                      'academics__program_percentage_social_science', 'student__demographics_veteran']
    for col in log_percentage:
        df['custom__log_{}'.format(col)] = df[col].map(lambda x: np.log(x*100+1))

    df = df.drop(log_cols, axis=1)
    df = df.drop(log_percentage, axis=1)

    df['custom__academics_program_percentage_health'] = df['academics__program_percentage_health'].replace(1, np.nan)
    
    df['custom__sat_0'] = df['admissions__sat_scores_average_by_ope_id'].fillna(0)
    df['custom__act_0'] = df['admissions__act_scores_midpoint_cumulative'].fillna(0)

    df = df.drop(list(df.filter(like='scores')), axis=1)
    #df.loc[:,list(df.filter(like='scores'))] = df.filter(like='scores').fillna(0)
    df = df.drop(list(df.filter(like='net_price_private')), axis=1)

    return df

In [5]:
def combine(df_num, df_cat):
    return pd.merge(df_num, df_cat, left_index=True, right_index=True)

In [6]:
def impute(train, test):
    avg = train.mean()
    train = train.fillna(avg)
    test = test.fillna(avg)
    return [train, test]

In [7]:
def cluster(train, test):
    aid_clusters = ['aid__cumulative_debt_75th_percentile', 
                'aid__cumulative_debt_90th_percentile',
                'aid__median_debt_completers_monthly_payments', 
                'aid__median_debt_completers_overall']
    cost_clusters = ['cost__attendance_academic_year',
                'cost__tuition_in_state']

    kmeans = KMeans(n_clusters=2, n_jobs=-1)
    
    kmeans.fit(train.loc[:, aid_clusters])
    train['custom__aid_clusters'] = kmeans.labels_ 
    test['custom__aid_clusters'] = kmeans.predict(test.loc[:,aid_clusters])

    kmeans.fit(train.loc[:, cost_clusters])
    train['custom__cost_clusters'] = kmeans.labels_ 
    test['custom__cost_clusters'] = kmeans.predict(test.loc[:,cost_clusters])
    
    return [train, test]

In [8]:
def transformer(train, test):
    def transform(df):
        df_num = split(df, numerical_features)
        df_cat = split(df, categorical_features)
        df_num = custom_num(df_num)
        df_cat = custom_cat(df_cat)
        df = combine(df_num, df_cat)
        return df
    train = transform(train)
    test = transform(test)
    train, test = impute(train, test)
    train, test = cluster(train, test)
    return [train, test]

In [9]:
x = pd.read_csv('train_values.csv', index_col='row_id')
test = pd.read_csv('test_values.csv', index_col='row_id')
y = pd.read_csv('train_labels.csv', index_col='row_id')

In [10]:
numerical_features = [i for i in x.columns if len(x[i].unique()) > 4 and x[i].dtype != 'O']
categorical_features = [i for i in x.columns if len(x[i].unique()) < 5 or x[i].dtype == 'O']

In [26]:
X_train, X_test, y_train, y_test = train_test_split(x, y['repayment_rate']) 
X_train, X_test = transformer(X_train, X_test)

xgb = XGBRegressor(n_estimators=1000, max_depth=8, nthread=4)
xgb.fit(X_train, y_train)
print('Training Set Score: {:.3f}'.format(xgb.score(X_train, y_train)))
print('Test Set Score: {:.3f}'.format(xgb.score(X_test, y_test)))
print('RMSE: {}'.format(np.sqrt(mean_squared_error(y_test, xgb.predict(X_test)))))

Training Set Score: 1.000
Test Set Score: 0.895
RMSE: 6.807678249801247


In [12]:
xgb = XGBRegressor(n_estimators=500, nthread=4)
param_grid = {'max_depth':[5, 8, 10, 13, 15]}
grid = GridSearchCV(xgb, param_grid=param_grid, cv=3, n_jobs=-1, verbose=3)
grid.fit(X_train, y_train)
print('Best CV Accuracy: {:.3f}'.format(grid.best_score_))
print('Training Set Score: {:.3f}'.format(grid.score(X_train, y_train)))
print('Test Set Score: {:.3f}'.format(grid.score(X_test, y_test)))
print('Best_Parameters: {}'.format(grid.best_params_))
print('RMSE: {}'.format(np.sqrt(mean_squared_error(y_test, grid.predict(X_test)))))

SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-12-59b7ecb1ee3e>, line 10)

In [ ]:
'''
X_train, X_test = transformer(x, test)
xgb.fit(X_train, y)
test['repayment_rate'] = xgb.predict(X_test)
test[['repayment_rate']].to_csv('results4.csv')
'''